In [ ]:
import numpy as np
import cv2 as cv
import imgkit
import cvzone
from cvzone.SelfiSegmentationModule import SelfiSegmentation

def process_video(input_imageboard_path, video_path,  background_image_path, output_video_path, wkhtmltoimage_path, html_text):
    output_imagetext_path="generated_image.png"
    # Initialiser le SelfiSegmentation
    segmentor = SelfiSegmentation()
    # Spécifiez le chemin correct vers wkhtmltoimage
    config = imgkit.config(wkhtmltoimage=wkhtmltoimage_path)

    def rgb_to_hex(rgb):
        """Convertit une couleur RGB en code hexadécimal."""
        return '#{:02x}{:02x}{:02x}'.format(rgb[0], rgb[1], rgb[2])

    # Charger l'image d'origine
    img_orig = cv.imread(input_imageboard_path)

    # Calculer la couleur moyenne de l'image
    average_color = np.mean(img_orig, axis=(0, 1))
    average_color = average_color.astype(int)

    # Convertir la couleur moyenne en format hexadécimal
    background_color = rgb_to_hex(average_color)

    # Définir le contenu HTML, en utilisant le texte passé en paramètre
    html = f"""
    <html>
      <body style="font-family: Arial, sans-serif; color: black; background-color: {background_color}; margin: 20px;">
        {html_text}
      </body>
    </html>
    """

    # Définir les dimensions de l'image générée
    width = int(img_orig.shape[1])
    height = int(img_orig.shape[0])

    # Convertir le HTML en image avec les dimensions spécifiées
    try:
        imgkit.from_string(html, output_imagetext_path, config=config, options={'width': width, 'height': height})
        print("Image générée avec succès.")
    except Exception as e:
        print(f"Erreur lors de la génération de l'image : {e}")

    # Charger l'image générée
    img_gen = cv.imread(output_imagetext_path)

    # Vérifiez que l'image générée existe
    if img_gen is None:
        raise FileNotFoundError("L'image générée n'a pas pu être chargée. Assurez-vous que la génération a réussi.")

    # Redimensionner img_gen pour correspondre à img_orig
    img_gen_resized = cv.resize(img_gen, (img_orig.shape[1], img_orig.shape[0]))

    # Superposer les deux images
    board_image = cv.addWeighted(img_orig, 0.4, img_gen_resized, 0.6, 0)
    

    # Initialiser la capture vidéo
    cap = cv.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Video file could not be opened.")
        return

    # Obtenir les propriétés vidéo
    w = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv.CAP_PROP_FPS))



    # Charger et redimensionner l'image de fond
    photo = cv.imread(background_image_path)
    photo = cv.resize(photo, (w, h))

    # Lire et redimensionner l'image source
    w1 = 440
    h1 = 300
    m1 = cv.resize(board_image, (w1, h1))

    # Ajuster la taille de la sous-région pour s'adapter à img1
    photo[204:204+300, 20:20+440] = m1

    # Définir le codec et initialiser VideoWriter pour enregistrer la vidéo
    fourcc = cv.VideoWriter_fourcc(*'mp4v')  # Codec pour .mp4
    out = cv.VideoWriter(output_video_path, fourcc, fps, (w, h))

    while True:
        suc, img = cap.read()
        if not suc:
            print("Error: Frame capture failed.")
            break

        # Supprimer l'arrière-plan
        imgOut = segmentor.removeBG(img, photo, cutThreshold=0.2)

        # Enregistrer le cadre dans le fichier vidéo
        out.write(imgOut)

        # Afficher les images empilées
        cv.imshow("Image stacked", imgOut)

        # Sortir de la boucle sur la touche 'q'
        if cv.waitKey(1) & 0xFF == ord('q'):
            break

    # Libérer les ressources
    cap.release()
    out.release()  # Libérer le fichier vidéo
    cv.destroyAllWindows()




In [ ]:
# Exemple d'utilisation de la fonction
html_content = """
<h1 style="text-align: center;">Machine Learning</h1>
<p style="font-size: 14px; margin: 20px;">
  Machine learning is a subset of artificial intelligence (AI) focused on building systems that can learn from and make decisions based on data. It involves creating algorithms and models that allow computers to improve their performance on a task over time without being explicitly programmed for each specific task.
</p>
"""

process_video(
    input_imageboard_path="board/1.jpeg",
    video_path="video/vid.mp4",
    background_image_path="board/background.png",
    output_video_path="output_video.mp4",
    # Téléchargez et installez "wkhtmltoimage".  https://wkhtmltopdf.org/downloads.html
    wkhtmltoimage_path='C:\\Program Files\\wkhtmltopdf\\bin\\wkhtmltoimage.exe',
    html_text=html_content
)